In [1]:
import pandas as pd
import pyrankvote
from pathlib import Path
from random import seed
seed(252)

In [2]:
df = pd.read_csv(Path('./data/20201219.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],Rank your movie choices. You don't have to do a full ranking. [Fourth Choice],Rank your movie choices. You don't have to do a full ranking. [Fifth Choice],Rank your movie choices. You don't have to do a full ranking. [Sixth Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice],What theme should we explore next week? [Fifth Choice],What theme should we explore next week? [Sixth Choice],What theme should we explore next week? [Seventh Choice]
0,12/15/2020 19:08:24,Daughters of the Dust (1991),Cat On A Hot Tin Roof (1958),Cold in July (2014),The Little Foxes (1941),Mud (2013),Pass the Ammo (1988),YOU MIGHT SAY THE CITY IS A CHARACTER IN THE M...,THE BEST OF MALE FRONTAL NUDITY,SAD MAN DOES HIS JOB,[Default to Nicolas Cage],"WHILE YOU WERE PARTYING, I STUDIED THE BLADE",RIPPED FROM THE BREADLINES: GREAT DEPRESSION-S...,ROOMMATES
1,12/16/2020 6:35:32,The Little Foxes (1941),Cold in July (2014),Cat On A Hot Tin Roof (1958),Pass the Ammo (1988),Daughters of the Dust (1991),NaN,SAD MAN DOES HIS JOB,"WHILE YOU WERE PARTYING, I STUDIED THE BLADE",THE BEST OF MALE FRONTAL NUDITY,RIPPED FROM THE BREADLINES: GREAT DEPRESSION-S...,[Default to Nicolas Cage],ROOMMATES,YOU MIGHT SAY THE CITY IS A CHARACTER IN THE M...
2,12/16/2020 9:05:36,Cold in July (2014),Mud (2013),Pass the Ammo (1988),Daughters of the Dust (1991),Cat On A Hot Tin Roof (1958),The Little Foxes (1941),RIPPED FROM THE BREADLINES: GREAT DEPRESSION-S...,"WHILE YOU WERE PARTYING, I STUDIED THE BLADE",SAD MAN DOES HIS JOB,ROOMMATES,THE BEST OF MALE FRONTAL NUDITY,YOU MIGHT SAY THE CITY IS A CHARACTER IN THE M...,[Default to Nicolas Cage]
3,12/16/2020 17:42:00,Cold in July (2014),Pass the Ammo (1988),Mud (2013),Daughters of the Dust (1991),NaN,NaN,"WHILE YOU WERE PARTYING, I STUDIED THE BLADE",YOU MIGHT SAY THE CITY IS A CHARACTER IN THE M...,RIPPED FROM THE BREADLINES: GREAT DEPRESSION-S...,SAD MAN DOES HIS JOB,[Default to Nicolas Cage],NaN,NaN
4,12/16/2020 17:42:34,Pass the Ammo (1988),Cold in July (2014),Daughters of the Dust (1991),Mud (2013),The Little Foxes (1941),Cat On A Hot Tin Roof (1958),YOU MIGHT SAY THE CITY IS A CHARACTER IN THE M...,THE BEST OF MALE FRONTAL NUDITY,"WHILE YOU WERE PARTYING, I STUDIED THE BLADE",SAD MAN DOES HIS JOB,[Default to Nicolas Cage],ROOMMATES,RIPPED FROM THE BREADLINES: GREAT DEPRESSION-S...
5,12/17/2020 9:22:40,Pass the Ammo (1988),Cat On A Hot Tin Roof (1958),Cold in July (2014),Daughters of the Dust (1991),The Little Foxes (1941),Mud (2013),[Default to Nicolas Cage],"WHILE YOU WERE PARTYING, I STUDIED THE BLADE",YOU MIGHT SAY THE CITY IS A CHARACTER IN THE M...,SAD MAN DOES HIS JOB,RIPPED FROM THE BREADLINES: GREAT DEPRESSION-S...,ROOMMATES,THE BEST OF MALE FRONTAL NUDITY


In [3]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [4]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['Daughters of the Dust (1991)', 'Cat On A Hot Tin Roof (1958)',
       'Cold in July (2014)', 'The Little Foxes (1941)', 'Mud (2013)',
       'Pass the Ammo (1988)'], dtype=object)

In [5]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [6]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [7]:
submission_dict

{'Daughters of the Dust (1991)': <Candidate('Daughters of the Dust (1991)')>,
 'Cat On A Hot Tin Roof (1958)': <Candidate('Cat On A Hot Tin Roof (1958)')>,
 'Cold in July (2014)': <Candidate('Cold in July (2014)')>,
 'The Little Foxes (1941)': <Candidate('The Little Foxes (1941)')>,
 'Mud (2013)': <Candidate('Mud (2013)')>,
 'Pass the Ammo (1988)': <Candidate('Pass the Ammo (1988)')>}

In [8]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [9]:
ballots

[<Ballot(Daughters of the Dust (1991), Cat On A Hot Tin Roof (1958), Cold in July (2014), The Little Foxes (1941), Mud (2013), Pass the Ammo (1988))>,
 <Ballot(The Little Foxes (1941), Cold in July (2014), Cat On A Hot Tin Roof (1958), Pass the Ammo (1988), Daughters of the Dust (1991))>,
 <Ballot(Cold in July (2014), Mud (2013), Pass the Ammo (1988), Daughters of the Dust (1991), Cat On A Hot Tin Roof (1958), The Little Foxes (1941))>,
 <Ballot(Cold in July (2014), Pass the Ammo (1988), Mud (2013), Daughters of the Dust (1991))>,
 <Ballot(Pass the Ammo (1988), Cold in July (2014), Daughters of the Dust (1991), Mud (2013), The Little Foxes (1941), Cat On A Hot Tin Roof (1958))>,
 <Ballot(Pass the Ammo (1988), Cat On A Hot Tin Roof (1958), Cold in July (2014), Daughters of the Dust (1991), The Little Foxes (1941), Mud (2013))>]

In [10]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [11]:
print(election_result)

ROUND 1
Candidate                       Votes  Status
----------------------------  -------  --------
Cold in July (2014)                 2  Hopeful
Pass the Ammo (1988)                2  Hopeful
Daughters of the Dust (1991)        1  Rejected
The Little Foxes (1941)             1  Rejected
Cat On A Hot Tin Roof (1958)        0  Rejected
Mud (2013)                          0  Rejected

FINAL RESULT
Candidate                       Votes  Status
----------------------------  -------  --------
Cold in July (2014)                 4  Elected
Pass the Ammo (1988)                2  Rejected
Daughters of the Dust (1991)        0  Rejected
The Little Foxes (1941)             0  Rejected
Cat On A Hot Tin Roof (1958)        0  Rejected
Mud (2013)                          0  Rejected



In [12]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

ROUND 1
Candidate                                                   Votes  Status
--------------------------------------------------------  -------  --------
YOU MIGHT SAY THE CITY IS A CHARACTER IN THE MOVIE              2  Hopeful
WHILE YOU WERE PARTYING, I STUDIED THE BLADE                    1  Hopeful
SAD MAN DOES HIS JOB                                            1  Hopeful
RIPPED FROM THE BREADLINES: GREAT DEPRESSION-SPLOITATION        1  Hopeful
[Default to Nicolas Cage]                                       1  Rejected
THE BEST OF MALE FRONTAL NUDITY                                 0  Rejected
ROOMMATES                                                       0  Rejected

ROUND 2
Candidate                                                   Votes  Status
--------------------------------------------------------  -------  --------
WHILE YOU WERE PARTYING, I STUDIED THE BLADE                    2  Hopeful
YOU MIGHT SAY THE CITY IS A CHARACTER IN THE MOVIE              2  Hopeful
SAD M